In [32]:
import pandas as pd
import numpy as np
import re
from datetime import datetime
from requests import get
from bs4 import BeautifulSoup
import lxml

In [105]:
#scrape
df = pd.DataFrame()
for i in range(2001,2020):    
    url = 'http://www.espn.com/nba/attendance/_/year/'+str(i)
    r = requests.get(url)
    soup = bs4.BeautifulSoup(r.text, 'lxml')

    table = soup.find(id="my-teams-table")
    rows = table.find_all('tr')

    rows = iter(rows)
    header_1 = [td.text for td in next(rows).find_all('td') if td.text]
    header_2 = [td.text for td in next(rows).find_all('td') if td.text]
    header = header_1[:2] + header_2 + header_1[-2:]
#     print(header)
    for row in rows:
        data = [td.text for td in row.find_all('td') if td.text]
        a = pd.Series(data)
        a = a.append(pd.Series(str(i)),ignore_index=True)
        df = df.append(a, ignore_index=True)

In [108]:
#rename columns
df_fin = df.rename(columns = {0:'Rk',1:'Team',2:'home_games',3:'home_total',4:'home_avg',5:'home_pct',
                     6:'road_games',7:'road_avg',8:'road_pct',
                     9:'ov_games',10:'ov_avg',11:'ov_pct',12:'year'})

In [115]:
len(df_fin)

584

In [121]:
#clean data
df_cleanrank = df_fin.drop(df_fin[df_fin['Rk'].str.contains('[a-zA-Z]', regex=True)].index.values,axis = 0)
df_cleanrank['Rk'] = df_cleanrank['Rk'].astype('int64')
df_cleanteam = df_cleanrank.drop(df_cleanrank[df_cleanrank['Rk']>30].index.values,axis = 0)
df_cleanteam

,Rk,Team,home_games,home_total,home_avg,home_pct,road_games,road_avg,road_pct,ov_games,ov_avg,ov_pct,year
0,1,Spurs,41,"913,175","22,272",--,41,"16,690",--,82,"19,481",--,2001
1,2,Bulls,41,"888,654","21,674",--,41,"16,336",--,82,"19,005",--,2001
2,3,Trail Blazers,41,"831,385","20,277",--,41,"17,037",--,82,"18,657",--,2001
3,4,NY Knicks,41,"810,283","19,763",--,41,"17,680",--,82,"18,721",--,2001
4,5,76ers,41,"805,692","19,651",--,41,"18,582",--,82,"19,116",--,2001
...,...,...,...,...,...,...,...,...,...,...,...,...,...
575,26,Grizzlies,41,"607,564","15,578",86.0,41,"17,579",93.3,82,"16,604",89.8,2019
576,27,Hawks,41,"628,440","15,327",81.8,41,"17,786",93.7,82,"16,557",87.8,2019
577,28,Timberwolves,41,"627,543","15,305",80.7,41,"17,641",93.7,82,"16,473",87.1,2019
578,29,Suns,41,"627,023","15,293",83.0,41,"17,982",95.3,82,"16,638",89.2,2019


In [122]:
df_cleanteam.to_csv('ESPN_Attendance.csv',index=False)